In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import mean, desc
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

#spark = SparkSession.builder.master("local[*]").getOrCreate()
#sc = SparkContext("spark://winemap:7077")
sparkSession = SparkSession.builder.master("spark://winemap:7077").getOrCreate()
#  .appName("my-spark-app")
#  .config("spark.some.config.option", "config-value")


In [ ]:
df = (sparkSession.read
         .format("com.databricks.spark.csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load("wine_no_des.csv"))
table = df.select('country','points').groupBy('country').agg(mean('points')).orderBy('avg(points)',ascending=False)
countryCols = table.select('country').collect()
countries = list()
for country in countryCols:
    countries.append(str(country[0]))
pointCols = table.select('avg(points)').collect()
points = list()
for point in pointCols:
    points.append(point[0])
data =  dict(type = 'choropleth',
        locationmode='country names',
        locations = countries,
        colorscale='Blues',
        z = points,
        colorbar = {'title': 'Average Rating'}
)
layout = dict(geo = {'scope':'world'})
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)